In [1]:
"""
Simple momentum-based model to predict daily excess returns and map them to allocation.

- Momentum idea:
  Use yesterday's excess return (lagged_market_forward_excess_returns)
  as today's predicted excess return.

- Then, linearly map the predicted excess return to an allocation in [0, 2]
  around a neutral allocation of 1.0.
"""

import os

import pandas as pd  # 남겨두어도 무방, 직접 사용은 안 함
import polars as pl

import kaggle_evaluation.default_inference_server


def excess_return_to_allocation(ret: float) -> float:
    """
    Map predicted excess return to an allocation in [0, 2].

    - Start from neutral allocation 1.0
    - Scale linearly by a factor k
    - Apply a small dead zone: if |ret| is very small, stay at 1.0
    - Clip strictly to [0, 2]

    This keeps the logic very simple while still reflecting a momentum-based signal.
    """
    # 아주 작은 초과수익률은 의미 없는 노이즈로 보고 트레이드 억제
    DEAD_ZONE = 0.0005
    if abs(ret) < DEAD_ZONE:
        return 1.0

    # 초과수익률을 포지션으로 선형 매핑하는 간단한 규칙
    k = 50.0  # scaling factor; small returns → small tilts around 1.0

    alloc = 1.0 + k * ret
    # Clip to [0, 2] as required by the competition
    if alloc < 0.0:
        alloc = 0.0
    elif alloc > 2.0:
        alloc = 2.0
    return float(alloc)


def predict(test: pl.DataFrame) -> float:
    """
    Momentum-based inference.

    1) From the provided single-row Polars DataFrame `test`,
       read the momentum signal:

       - Preferred: `lagged_market_forward_excess_returns`
         (yesterday's market excess return)

       - Fallback: `lagged_forward_returns - risk_free_rate`
         if `lagged_market_forward_excess_returns` is not available.

       - If no momentum column is available, fall back to 0.0
         (neutral prediction → allocation = 1.0)

    2) Convert this predicted excess return into an allocation in [0, 2].
    """
    if not isinstance(test, pl.DataFrame):
        raise TypeError("predict(test): expected Polars DataFrame as input")

    if test.height != 1:
        raise ValueError(
            f"predict(test): expected a single-row Polars DataFrame, got {test.height} rows"
        )

    cols = set(test.columns)

    # 1) 가장 직접적인 모멘텀: 전일 시장 초과수익률
    if "lagged_market_forward_excess_returns" in cols:
        # Polars에서 단일 값 읽기
        val = test.select("lagged_market_forward_excess_returns").to_series().item()
        if val is None:
            pred_ret = 0.0
        else:
            pred_ret = float(val)

    # 2) 대체 모멘텀: 전일 forward_returns - risk_free_rate (있을 경우)
    elif "lagged_forward_returns" in cols and "risk_free_rate" in cols:
        lag_val = test.select("lagged_forward_returns").to_series().item()
        rf_val = test.select("risk_free_rate").to_series().item()
        lag_val = 0.0 if lag_val is None else float(lag_val)
        rf_val = 0.0 if rf_val is None else float(rf_val)
        pred_ret = lag_val - rf_val

    # 3) 어떤 모멘텀 정보도 없으면 중립 예측
    else:
        pred_ret = 0.0

    # 예측된 초과수익률을 allocation으로 변환
    alloc = excess_return_to_allocation(pred_ret)
    return alloc


# When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting
# or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very
# first `predict` call, which does not have the usual 1 minute response deadline.
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(
    predict
)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(("/kaggle/input/hull-tactical-market-prediction/",))
